In [1]:
import pandas as pd
import numpy as np 
from PIL import Image
from torch.nn import Conv2d, Linear, LazyLinear
import torch
import torch.nn as nn
import os
import warnings
from torchsummary import summary
warnings.filterwarnings("ignore")

In [2]:
# WRAP EVERY CONV WITH A RELU LAYER
class SubWayNet(nn.Module):

    def __init__(self,  num_classes=5):

        super(SubWayNet,self).__init__()

        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3,stride=1)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)

        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv_layer3 =  nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3,stride=1)
        self.conv_layer4 =  nn.Conv2d(in_channels=128, out_channels=256,kernel_size=3)

        self.max_pool2 =  nn.MaxPool2d(kernel_size=2, stride=2)
    

        # self.fc1 =  LazyLinear(out_features=64)
        self.fc1 = nn.Linear(56*120*256,64)
        self.relu1 =  nn.ReLU()
        self.fc2 =  nn.Linear(64, num_classes) 

    def forward(self, x):

        out =  self.conv_layer1(x) 
        out = self.conv_layer2(out) 
        out =  self.max_pool1(out)

        out =  self.conv_layer3(out)
        out = self.conv_layer4(out)
        out =  self.max_pool2(out) 
        # print(out.shape)
        out =  out.view(-1,56*120*256)

        out =  self.fc1(out)
        out =  self.relu1(out)
        out =  self.fc2(out) 


        return out 

        



In [3]:
surf = SubWayNet()

In [4]:
class conv(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 =  nn.Conv2d(3,6,3,1)
        self.conv2 =  nn.Conv2d(6,16,3,1)
        self.fc1 = LazyLinear(120)
        self.fc2= nn.Linear(120,84)
        self.fc3 =  nn.Linear(84,2)
        self.maxpool =  nn.MaxPool2d(kernel_size=2, stride=1)
        self.maxpool2 =  nn.MaxPool2d(kernel_size=2,stride=1)
        self.relu = nn.ReLU()

    def forward(self,x):
        x = self.relu(self.conv1(x))
        x =  self.maxpool(x)
        x = self.relu(self.conv2(x))
        x =  self.maxpool2(x)
        x =  x.view(-1,54*54*16)
        x =  self.relu(self.fc1(x))
        x =  self.relu(self.fc2(x))
        x =  self.fc3(x)

        return x
        

In [5]:
torch.empty(1, 3, 32, 32).size(-1)

32

In [6]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset, random_split, DataLoader


In [7]:
# fn = pic_dir+str(df_copy.image.iloc[3])+'.jpg'

In [8]:
# image = Image.open(fn)
# transform = transforms.ToTensor()
# image_tensor = transform(image)

# inverted_image_tensor = torch.flip(image_tensor, dims=[2])
# inverted_image = transforms.ToPILImage()(inverted_image_tensor)



In [9]:

class LoadImages(Dataset):

    def __init__(self, csv_file='testing.csv', transform=None):
        self.root_dir =  "SHOTS" 
        self.re_map = {'a':"left", "w":"jump","s":"roll","d":"right",'blank':'None'}
        
        self.df=  pd.read_csv(csv_file,dtype={'image':int, 'keyPress':str})
        
        self.df['keyPress'] = self.df["keyPress"].apply(self.cleaner)
        self.df = self.df[self.df.image.apply(self.exists)]

        # self.df['oneHot'] = torch.tensor(self.df['keyPress'].astype('category').cat.codes.values)
        

        self.df["keyPress"] = self.df["keyPress"].apply(lambda x: self.re_map.get(x))
        
        self.one_hot_encoded_df = pd.get_dummies(self.df['keyPress'], dtype=float)
        self.df = pd.concat([self.df, self.one_hot_encoded_df], axis=1)

        # self.transform = transform
    
    
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        pt =  os.path.join(self.root_dir,str(self.df.iloc[idx][0])+".jpg")
        img = Image.open(pt)
        # y_label =  torch.tensor(self.df['oneHot'].iloc[idx])
        y_label =  self.df.drop(columns=['image','keyPress']).iloc[idx]
        # y_label =  y_label.view(-1)
        # print(y_label)
        transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((int(img.size[0] / 2), int(img.size[1] / 2)))
        ])
        
        # if self.transform:
        img =  transform(img)

        return img, torch.tensor(y_label).view(-1)
        
    def getMappings(self):
        return dict(enumerate(self.df['keyPress'].astype('category').cat.categories))

    def cleaner(self,x):
        if 'blank' not in x:
            
            return  x[1]
        else:
            return 'blank'
            
            
    def exists(self,filename:str):
        if os.path.exists(f"SHOTS/{filename}.jpg"):
            return True
        else:
        
            return False

        



In [104]:
def prepare(filename:str):
    
    pt =  f"SHOTS/{filename}.jpg"
    img = Image.open(pt)
    
        # y_label =  y_label.view(-1)
        # print(y_label)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((int(img.size[0] / 2), int(img.size[1] / 2)))
    ])
        
        # if self.transform:
    return  transform(img).unsqueeze(0).to(device)

In [89]:
custom.getMappings()

{0: 'None', 1: 'jump', 2: 'left', 3: 'right', 4: 'roll'}

In [101]:
a1 =  SubWayNet()

In [103]:
aa = a1.load_state_dict(previous)

TypeError: Expected state_dict to be dict-like, got <class 'method'>.

In [105]:
prepare(600)

tensor([[[[0.4901, 0.4862, 0.4843,  ..., 0.2353, 0.2353, 0.2353],
          [0.4671, 0.4641, 0.4644,  ..., 0.2345, 0.2349, 0.2351],
          [0.4618, 0.4617, 0.4644,  ..., 0.2205, 0.2300, 0.2358],
          ...,
          [0.1974, 0.1930, 0.2075,  ..., 0.4657, 0.4668, 0.4656],
          [0.2266, 0.2284, 0.2283,  ..., 0.4611, 0.4632, 0.4608],
          [0.1905, 0.2017, 0.1941,  ..., 0.3929, 0.3941, 0.3928]],

         [[0.4544, 0.4555, 0.4564,  ..., 0.2549, 0.2549, 0.2549],
          [0.4657, 0.4673, 0.4674,  ..., 0.2541, 0.2545, 0.2547],
          [0.4683, 0.4683, 0.4675,  ..., 0.2401, 0.2496, 0.2554],
          ...,
          [0.1893, 0.1871, 0.1996,  ..., 0.1553, 0.1543, 0.1555],
          [0.2237, 0.2284, 0.2257,  ..., 0.1562, 0.1546, 0.1564],
          [0.2042, 0.2160, 0.2078,  ..., 0.1492, 0.1484, 0.1493]],

         [[0.5059, 0.5059, 0.5059,  ..., 0.3294, 0.3294, 0.3294],
          [0.5059, 0.5059, 0.5059,  ..., 0.3286, 0.3290, 0.3292],
          [0.5059, 0.5059, 0.5059,  ..., 0

In [107]:
f1 =  prepare('600').to(device)
model.eval()
with torch.no_grad():
    output = model(f1)
torch.max(output,1)[1]

tensor([0], device='cuda:0')

In [80]:
output.max(1)

torch.return_types.max(
values=tensor([1.5712], device='cuda:0'),
indices=tensor([0], device='cuda:0'))

In [10]:
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize((int(original_size[0] / 2), int(original_size[1] / 2)))
# ])

In [11]:

custom =  LoadImages(csv_file="testing.csv" )

In [77]:
custom

In [13]:
custom.getMappings()

{0: 'None', 1: 'jump', 2: 'left', 3: 'right', 4: 'roll'}

In [14]:
device =  torch.device("cuda" if torch.cuda.is_available else 'cpu')

In [15]:
train_size  =  int(0.8 * len(custom))
test_size=  int(len(custom) - train_size)

In [16]:
train_dataset, test_dataset =  random_split(custom, [train_size,test_size])

In [17]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [55]:
model = surf.to(device)

In [56]:
torch.save(model.state_dict,'SUBWAY.pt')

In [57]:
criterion =  nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [59]:
num_epoch =  1
losses=[] 
for epoch in range(num_epoch):
    

    for batch_idx, (data,target) in enumerate(train_loader):
        
        data =  data.to(device)
        target = target.to(device)
        # image = image.view(image.size(0), -1)
        # feed forward 
        logits =  model(data)
        loss =  criterion(logits,target)
        
        # store the losses
        losses.append(loss.item()) 

        # backward propagation
        optimizer.zero_grad()
        loss.backward()

        # step
        optimizer.step()
        if batch_idx%100 ==0:
            print(f'Cost at epoch {epoch}  is {sum(losses)/len(losses)}')
    

Cost at epoch 0  is 0.7300033122301102
Cost at epoch 0  is 0.9479775810890859
Cost at epoch 0  is 0.9009317611320992
Cost at epoch 0  is 0.8842985322853656
Cost at epoch 0  is 0.8693902261945375
Cost at epoch 0  is 0.8674932275926639
Cost at epoch 0  is 0.860658856750328
Cost at epoch 0  is 0.859311437752422
Cost at epoch 0  is 0.8568360245694754
Cost at epoch 0  is 0.8472061750337432
Cost at epoch 0  is 0.8446291833818257
Cost at epoch 0  is 0.8472226218072584
Cost at epoch 0  is 0.8447070659156097
Cost at epoch 0  is 0.8443875903296617
Cost at epoch 0  is 0.8434603071208514
Cost at epoch 0  is 0.8458116807683875
Cost at epoch 0  is 0.8464363238504907
Cost at epoch 0  is 0.8416062122641207
Cost at epoch 0  is 0.8411982548730794
Cost at epoch 0  is 0.840356315759014
Cost at epoch 0  is 0.8387774328651397
Cost at epoch 0  is 0.8362983562768583


In [93]:
previous =  torch.load("SUBWAY.pt")

In [95]:
previous

<bound method Module.state_dict of SubWayNet(
  (conv_layer1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1720320, out_features=64, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=64, out_features=5, bias=True)
)>

In [ ]:
i.shape, j.shape

In [ ]:
downsampled_x = losses[::int(len(losses)/200)]


In [ ]:
plt.plot(range(200), downsampled_x_list)

In [ ]:
model.parameters

In [49]:
def check_accuracy(loader,model):
    num_correct = 0
    num_samples=0
    model.eval()

    with torch.no_grad():
        for x, y in test_loader:
            x= x.to(device)
            y= y.to(device)

            scores = model(x)
            _, predictions =  scores.max(1)
            num_correct += (predictions==y.view(-1,1)).sum()
            num_samples += predictions.size(0)
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/ float(num_samples) *100}')
        
        

In [50]:
check_accuracy(train_loader,model)

Got 138640 / 4334 with accuracy 3198.8924780802954


In [60]:
import torch

def calculate_accuracy(model, data_loader):
    # Set the model to evaluation mode
    model.eval()
    
    # Initialize variables to store total correct predictions and total samples
    total_correct = 0
    total_samples = 0
    
    # Iterate over the validation dataset
    with torch.no_grad():
        for inputs, labels in data_loader:
            # Forward pass: compute predicted outputs by passing inputs to the model
            inputs,labels =  inputs.to(device), labels.to(device).view(-1,1)
            outputs = model(inputs)
            
            # Get the predicted class labels
            _, predicted = torch.max(outputs, 1)
            
            # Compute the number of correct predictions in the current batch
            batch_correct = (predicted == labels).sum().item()
            
            # Update total correct predictions and total samples
            total_correct += batch_correct
            total_samples += labels.size(0)
    
    # Calculate the accuracy
    accuracy = total_correct / total_samples
    
    # Convert accuracy to percentage
    accuracy_percentage = accuracy * 100
    
    return accuracy_percentage

# Assuming `model` is your PyTorch model
# Assuming `valid_loader` is your data loader for the validation set

# Calculate the accuracy of the model on the validation set
accuracy_percentage = calculate_accuracy(model, train_loader)

print('Validation Accuracy:', accuracy_percentage, '%')


Validation Accuracy: 640.0 %


In [63]:
accuracy_percentage/100

6.4

In [33]:
sc = model(i.to(device))

In [54]:
640/100

6.4

In [47]:
pred.size(0)

8

In [44]:
j.to(device).view(-1,1).shape

torch.Size([40, 1])

In [45]:
(pred==j.to(device).view(-1,1))

tensor([[False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ 

In [ ]:
 x.view( -1, x.size( 1 )).shape

In [ ]:
sc = model(x.to(device))
sc.max(1)

In [ ]:
pip install torchsummary

In [ ]:
from torchsummary import summary

In [ ]:
summary(surf.to(device))

In [ ]:
df=  pd.read_csv("testing.csv",dtype={'image':int, 'keyPress':str})


In [ ]:
 # # self.re_map = {'a':"left", "w":"jump","s":"roll","d":"right",'blank':'None'}
        
 #        self.df=  pd.read_csv(csv_file,dtype={'image':int, 'keyPress':str})
        
 #        self.df['keyPress'] = self.df["keyPress"].apply(self.cleaner)
 #        self.df = self.df[~self.df.image.apply(self.exists).isna()]

 #        self.df['oneHot'] = torch.tensor(self.df['keyPress'].astype('category').cat.codes.values)

 #        self.df["keyPress"] = self.df["keyPress"].apply(lambda x: self.re_map.get(x))

In [ ]:
def cleaner(x):
    if 'blank' not in x:
        return  x[1]
    else:
        return 'blank'
            # 
def exists(filename:str):
    if os.path.exists(f"SHOTS/{filename}.jpg"):
        return True
    else:
        
        return False

In [ ]:
# df=pd.read_csv("testing.csv",dtype={'image':int, 'keyPress':str})
# df['keyPress']  = df["keyPress"].apply(cleaner)
ss = df.image.apply(exists)


In [ ]:
os.path.exists(f"SHOTS/{df.image.iloc[1]}.jpg")

In [ ]:
df[df.image.apply(exists)].shape

In [ ]:
df.shape, ss.shape

In [ ]:
exists('2708.jpg')

In [ ]:
df

In [ ]:
# one_hot_encoded_df = pd.get_dummies(df['keyPress'], dtype=int)


In [ ]:
# one_hot_encoded_df

In [ ]:
# df_encoded = pd.concat([df, one_hot_encoded_df], axis=1)


In [ ]:
# samp = df_encoded.drop(columns='keyPress')

In [ ]:
# df_encoded[['a','blank','d','s','w']]